In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
full_df = pd.read_csv(open('../davis_all_pairs.csv','r'))

In [3]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('../data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [4]:
def create_davis_test_train(test_fold_number,all_9_folds):
    test_protein_fold_id = test_fold_number[0]
    test_ligand_fold_id = test_fold_number[1]
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
            if str(i) == test_protein_fold_id:
                test_set = pd.concat([test_set, df.copy()], ignore_index=True)
                
            if str(i) != test_protein_fold_id:
                print("train ->",i, j)
                train_set = pd.concat([train_set, df.copy()], ignore_index=True)
                
                
    return train_set, test_set


# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [5]:
fold_number = '0x'

In [6]:
train, test = create_davis_test_train(test_fold_number=fold_number, all_9_folds=all_9_folds)

train -> 1 0
train -> 1 1
train -> 1 2
train -> 2 0
train -> 2 1
train -> 2 2


In [7]:
train

,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CCC1C(=O)N(C2=CN=C(N=C2N1C3CCCC3)NC4=C(C=C(C=C...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,5.000000,"[C, C, C, 1, C, (, =, O, ), N, (, C, 2, =, C, ...","[M, V, D, G, V, M, I, L, P, V, L, I, M, I, A, ..."
1,CN1C2=C(C=C(C=C2)OC3=CC(=NC=C3)C4=NC=C(N4)C(F)...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,5.000000,"[C, N, 1, C, 2, =, C, (, C, =, C, (, C, =, C, ...","[M, V, D, G, V, M, I, L, P, V, L, I, M, I, A, ..."
2,C1CC1CONC(=O)C2=C(C(=C(C=C2)F)F)NC3=C(C=C(C=C3...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,5.000000,"[C, 1, C, C, 1, C, O, N, C, (, =, O, ), C, 2, ...","[M, V, D, G, V, M, I, L, P, V, L, I, M, I, A, ..."
3,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,5.468521,"[C, C, 1, =, C, C, 2, =, C, (, N, 1, ), C, =, ...","[M, V, D, G, V, M, I, L, P, V, L, I, M, I, A, ..."
4,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,6.207608,"[C, C, 1, =, C, (, C, (, =, C, C, =, C, 1, ), ...","[M, V, D, G, V, M, I, L, P, V, L, I, M, I, A, ..."
...,...,...,...,...,...
19987,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, N, 1, C, =, N, C, 2, =, C, 1, C, =, C, (, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
19988,CC(C)S(=O)(=O)C1=CC=CC=C1NC2=NC(=NC=C2Cl)NC3=C...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, C, (, C, ), S, (, =, O, ), (, =, O, ), C, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
19989,C1=CC(=CC(=C1)O)C2=NC3=C(N=C2C4=CC(=CC=C4)O)N=...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, 1, =, C, C, (, =, C, C, (, =, C, 1, ), O, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
19990,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.221849,"[C, C, 1, =, C, N, =, C, (, N, =, C, 1, N, C, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."


In [8]:
test

,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CCC1C(=O)N(C2=CN=C(N=C2N1C3CCCC3)NC4=C(C=C(C=C...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000,"[C, C, C, 1, C, (, =, O, ), N, (, C, 2, =, C, ...","[M, S, D, V, A, I, V, K, E, G, W, L, H, K, R, ..."
1,CN1C2=C(C=C(C=C2)OC3=CC(=NC=C3)C4=NC=C(N4)C(F)...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000,"[C, N, 1, C, 2, =, C, (, C, =, C, (, C, =, C, ...","[M, S, D, V, A, I, V, K, E, G, W, L, H, K, R, ..."
2,C1CC1CONC(=O)C2=C(C(=C(C=C2)F)F)NC3=C(C=C(C=C3...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000,"[C, 1, C, C, 1, C, O, N, C, (, =, O, ), C, 2, ...","[M, S, D, V, A, I, V, K, E, G, W, L, H, K, R, ..."
3,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000,"[C, C, 1, =, C, C, 2, =, C, (, N, 1, ), C, =, ...","[M, S, D, V, A, I, V, K, E, G, W, L, H, K, R, ..."
4,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000,"[C, C, 1, =, C, (, C, (, =, C, C, =, C, 1, ), ...","[M, S, D, V, A, I, V, K, E, G, W, L, H, K, R, ..."
...,...,...,...,...,...
10059,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,MAHLRGFANQHSRVDPEELFTKLDRIGKGSFGEVYKGIDNHTKEVV...,5.000000,"[C, N, 1, C, =, N, C, 2, =, C, 1, C, =, C, (, ...","[M, A, H, L, R, G, F, A, N, Q, H, S, R, V, D, ..."
10060,CC(C)S(=O)(=O)C1=CC=CC=C1NC2=NC(=NC=C2Cl)NC3=C...,MAHLRGFANQHSRVDPEELFTKLDRIGKGSFGEVYKGIDNHTKEVV...,5.113509,"[C, C, (, C, ), S, (, =, O, ), (, =, O, ), C, ...","[M, A, H, L, R, G, F, A, N, Q, H, S, R, V, D, ..."
10061,C1=CC(=CC(=C1)O)C2=NC3=C(N=C2C4=CC(=CC=C4)O)N=...,MAHLRGFANQHSRVDPEELFTKLDRIGKGSFGEVYKGIDNHTKEVV...,5.000000,"[C, 1, =, C, C, (, =, C, C, (, =, C, 1, ), O, ...","[M, A, H, L, R, G, F, A, N, Q, H, S, R, V, D, ..."
10062,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,MAHLRGFANQHSRVDPEELFTKLDRIGKGSFGEVYKGIDNHTKEVV...,5.142668,"[C, C, 1, =, C, N, =, C, (, N, =, C, 1, N, C, ...","[M, A, H, L, R, G, F, A, N, Q, H, S, R, V, D, ..."


# To ensure that there are no common targets or drugs in train and test


In [9]:
test_smiles = list(test['SMILES'])
test_targets = list(test['Target Sequence'])
train_smiles = list(train['SMILES'])
train_targets = list(train['Target Sequence'])

# for i in test_smiles:
#     if i in train_smiles:
#         print("common entity present")
for i in test_targets:
    if i in train_targets:
        print("common entity present")


# Creating similarity matrices for this fold

In [10]:
import rdkit
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import FingerprintSimilarity as fs
from rdkit.Chem.Fingerprints import FingerprintMols
from Bio import pairwise2

In [11]:
train_targets = list(set(list(train['Target Sequence'])))
train_smiles = list(set(list(train['SMILES'])))

def computeLigandSimilarity(smiles):
    fingerprints = {}
    for smile in smiles:
        mol = AllChem.MolFromSmiles(smile)
        if mol == None:
            mol = AllChem.MolFromSmiles(smile, sanitize=False)
        fp = FingerprintMols.FingerprintMol(mol)
        fingerprints[smile] = fp
    
    n = len(smiles)
    sims = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1):
            fpi = fingerprints[smiles[i]]
            fpj = fingerprints[smiles[j]]
            sim = fs(fpi, fpj)
            sims[i, j] = sims[j, i] = sim
    return sims

def computeProteinSimilarity(targets):
    n = len(targets)
    mat = np.zeros((n,n))
    mat_i = np.zeros(n)
    for i in range(n):
        seq = targets[i]
        s = pairwise2.align.localxx(seq,seq, score_only=True)
        mat_i[i] = s
        
    for i in range(n):
        print(i)
        for j in range(n):
            if mat[i][j] == 0 :
                s1 = targets[i]
                s2 = targets[j]
                sw_ij = pairwise2.align.localxx(s1,s2,score_only=True)
                normalized_score = sw_ij /math.sqrt(mat_i[i]*mat_i[j])
                mat[i][j] = mat[j][i] = normalized_score
    
    return mat

In [12]:
ligand_similarity_matrix = computeLigandSimilarity(train_smiles)

In [13]:
np.shape(ligand_similarity_matrix)

(68, 68)

In [14]:
print(len(train_targets))
protein_similarity_matrix = computeProteinSimilarity(train_targets)

237
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236


In [15]:
np.shape(protein_similarity_matrix)

(237, 237)

In [16]:
LSM = ligand_similarity_matrix
PSM = protein_similarity_matrix

# creating outer products for train set

In [17]:
outer_train_prods = []
for i,row in train.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = train_targets.index(seq)
    smi_id = train_smiles.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_train_prods.append([ki_x_kj])
outer_train_prods = np.array(outer_train_prods)
print(np.shape(outer_train_prods))

(19992, 1, 68, 237)


# Creating similarity matrcies for test set

In [18]:
test_targets = list(set(list(test['Target Sequence'])))
test_smiles = list(set(list(test['SMILES'])))

In [19]:
test_PSM = np.zeros((len(test_targets), len(train_targets)))
np.shape(test_PSM)

(142, 237)

In [20]:
s_train_PSM = np.zeros(len(train_targets))
s_test_PSM = np.zeros(len(test_targets))

for i in range(len(train_targets)):
    seq = train_targets[i]
    s_train_PSM[i] = pairwise2.align.localxx(seq,seq, score_only=True)
    
for i in range(len(test_targets)):
    seq = test_targets[i]
    s_test_PSM[i] = pairwise2.align.localxx(seq,seq, score_only=True)
    
for i in range(len(test_targets)):
    print(i)
    for j in range(len(train_targets)):
        seq1 = test_targets[i]
        seq2 = train_targets[j]
        s_ij=pairwise2.align.localxx(seq1, seq2, score_only=True)
        N_S = s_ij / math.sqrt(s_train_PSM[j] * s_test_PSM[i])
        test_PSM[i][j] = N_S

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141


In [21]:
test_LSM = np.zeros((len(test_smiles), len(train_smiles)))
np.shape(test_LSM)

(68, 68)

In [22]:
for i in range(len(test_smiles)):
    print(i)
    for j in range(len(train_smiles)):
        smi1 = test_smiles[i]
        smi2 = train_smiles[j]
        
        mol1 = AllChem.MolFromSmiles(smi1)
        if mol1 == None:
            mol1= AllChem.MolFromSmiles(smi1, sanitize=False)
        fp1 = FingerprintMols.FingerprintMol(mol1)
        
        mol2 = AllChem.MolFromSmiles(smi2)
        if mol2 == None:
            mol2= AllChem.MolFromSmiles(smi2, sanitize=False)
        fp2 = FingerprintMols.FingerprintMol(mol2)
        
        test_LSM[i][j] = fs(fp1,fp2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


# Creating outer products for test set

In [23]:
outer_test_prods = []
for i,row in test.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = test_targets.index(seq)
    smi_id = test_smiles.index(smi)
    ki=test_LSM[smi_id]
    kj=test_PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_test_prods.append([ki_x_kj])
outer_test_prods = np.array(outer_test_prods)
print(np.shape(outer_test_prods))

(10064, 1, 68, 237)


In [24]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [25]:
device

device(type='cuda', index=0)

In [26]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, outer_prods, transform=None):
#         self.df = pd.read_csv(open(csv_file))
        self.df = dataframe
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [27]:
train_dataset = custom_dataset(dataframe = train, outer_prods = outer_train_prods)
test_dataset = custom_dataset(dataframe = test, outer_prods = outer_test_prods)


In [28]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [29]:
print(len(train_loader)*32, len(test_loader)*32)

20000 10080


In [31]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*57, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*15*57)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

torch.Size([32, 32, 64, 233])
torch.Size([32, 32, 32, 116])
torch.Size([32, 18, 30, 114])
torch.Size([32, 18, 15, 57])
torch.Size([32, 15390])
torch.Size([32, 15390])
torch.Size([32, 128])
torch.Size([32, 1])


In [32]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*57, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*57)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [33]:
model = ConvNet().to(device)

In [34]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics

In [35]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

In [36]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            total_preds = np.concatenate([total_preds, outputs])
            total_labels = np.concatenate([total_labels, labels])
    
    model.train()
    return total_labels, total_preds

# Train the model


In [37]:
model_file_name = 'best_sim-CNN-DTA_davis_fold' + fold_number +  '.model'
result_file_name = 'best_result_sim-CNNDTA_davis_fold'+fold_number + '.csv'

In [38]:
# Train the model
best_mse = 1000
best_ci = 0

total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
    G,P = predicting(model, device, test_loader)
    ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
    if ret[1] < best_mse:
        torch.save(model.state_dict(), model_file_name)
        with open(result_file_name, 'w') as f:
            f.write(','.join(map(str, ret)))
        best_epoch = epoch+1
        best_mse = ret[1]
        best_ci = ret[-1]
        best_r = ret[2]
        
        print('rmse improved at epoch ', best_epoch,
                      '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

Epoch [1/20], Step [1/625], Loss: 32.0389
Epoch [1/20], Step [2/625], Loss: 20.3438
Epoch [1/20], Step [3/625], Loss: 10.2855
Epoch [1/20], Step [4/625], Loss: 0.5626
Epoch [1/20], Step [5/625], Loss: 8.7517
Epoch [1/20], Step [6/625], Loss: 7.8067
Epoch [1/20], Step [7/625], Loss: 3.0468
Epoch [1/20], Step [8/625], Loss: 1.0663
Epoch [1/20], Step [9/625], Loss: 1.9085
Epoch [1/20], Step [10/625], Loss: 2.8839
Epoch [1/20], Step [11/625], Loss: 3.4040
Epoch [1/20], Step [12/625], Loss: 2.4660
Epoch [1/20], Step [13/625], Loss: 2.4312
Epoch [1/20], Step [14/625], Loss: 1.6467
Epoch [1/20], Step [15/625], Loss: 0.6982
Epoch [1/20], Step [16/625], Loss: 1.0361
Epoch [1/20], Step [17/625], Loss: 1.4573
Epoch [1/20], Step [18/625], Loss: 1.9567
Epoch [1/20], Step [19/625], Loss: 2.0995
Epoch [1/20], Step [20/625], Loss: 1.7849
Epoch [1/20], Step [21/625], Loss: 1.3739
Epoch [1/20], Step [22/625], Loss: 0.7942
Epoch [1/20], Step [23/625], Loss: 1.3036
Epoch [1/20], Step [24/625], Loss: 2.527

Epoch [1/20], Step [195/625], Loss: 0.5608
Epoch [1/20], Step [196/625], Loss: 1.1187
Epoch [1/20], Step [197/625], Loss: 0.6257
Epoch [1/20], Step [198/625], Loss: 0.4112
Epoch [1/20], Step [199/625], Loss: 0.8281
Epoch [1/20], Step [200/625], Loss: 0.6353
Epoch [1/20], Step [201/625], Loss: 1.2500
Epoch [1/20], Step [202/625], Loss: 0.6903
Epoch [1/20], Step [203/625], Loss: 0.7926
Epoch [1/20], Step [204/625], Loss: 0.5378
Epoch [1/20], Step [205/625], Loss: 0.8723
Epoch [1/20], Step [206/625], Loss: 0.4305
Epoch [1/20], Step [207/625], Loss: 0.7632
Epoch [1/20], Step [208/625], Loss: 1.5785
Epoch [1/20], Step [209/625], Loss: 0.4499
Epoch [1/20], Step [210/625], Loss: 0.9055
Epoch [1/20], Step [211/625], Loss: 1.5427
Epoch [1/20], Step [212/625], Loss: 0.7769
Epoch [1/20], Step [213/625], Loss: 1.5199
Epoch [1/20], Step [214/625], Loss: 0.9599
Epoch [1/20], Step [215/625], Loss: 0.7952
Epoch [1/20], Step [216/625], Loss: 0.9351
Epoch [1/20], Step [217/625], Loss: 2.0795
Epoch [1/20

Epoch [1/20], Step [390/625], Loss: 0.4861
Epoch [1/20], Step [391/625], Loss: 0.7443
Epoch [1/20], Step [392/625], Loss: 0.7345
Epoch [1/20], Step [393/625], Loss: 0.6507
Epoch [1/20], Step [394/625], Loss: 0.5858
Epoch [1/20], Step [395/625], Loss: 0.9551
Epoch [1/20], Step [396/625], Loss: 1.3156
Epoch [1/20], Step [397/625], Loss: 0.8496
Epoch [1/20], Step [398/625], Loss: 1.2359
Epoch [1/20], Step [399/625], Loss: 0.7631
Epoch [1/20], Step [400/625], Loss: 1.2500
Epoch [1/20], Step [401/625], Loss: 0.5239
Epoch [1/20], Step [402/625], Loss: 1.0278
Epoch [1/20], Step [403/625], Loss: 1.7399
Epoch [1/20], Step [404/625], Loss: 0.6249
Epoch [1/20], Step [405/625], Loss: 0.6323
Epoch [1/20], Step [406/625], Loss: 1.3583
Epoch [1/20], Step [407/625], Loss: 1.9996
Epoch [1/20], Step [408/625], Loss: 0.5858
Epoch [1/20], Step [409/625], Loss: 0.3471
Epoch [1/20], Step [410/625], Loss: 0.5820
Epoch [1/20], Step [411/625], Loss: 0.7160
Epoch [1/20], Step [412/625], Loss: 0.8024
Epoch [1/20

Epoch [1/20], Step [585/625], Loss: 0.7619
Epoch [1/20], Step [586/625], Loss: 0.3059
Epoch [1/20], Step [587/625], Loss: 0.3526
Epoch [1/20], Step [588/625], Loss: 1.3936
Epoch [1/20], Step [589/625], Loss: 0.6292
Epoch [1/20], Step [590/625], Loss: 1.2491
Epoch [1/20], Step [591/625], Loss: 0.9838
Epoch [1/20], Step [592/625], Loss: 1.1754
Epoch [1/20], Step [593/625], Loss: 1.0061
Epoch [1/20], Step [594/625], Loss: 0.5793
Epoch [1/20], Step [595/625], Loss: 1.3988
Epoch [1/20], Step [596/625], Loss: 0.3236
Epoch [1/20], Step [597/625], Loss: 1.4980
Epoch [1/20], Step [598/625], Loss: 0.9221
Epoch [1/20], Step [599/625], Loss: 0.6555
Epoch [1/20], Step [600/625], Loss: 1.0787
Epoch [1/20], Step [601/625], Loss: 0.8258
Epoch [1/20], Step [602/625], Loss: 0.5446
Epoch [1/20], Step [603/625], Loss: 0.4598
Epoch [1/20], Step [604/625], Loss: 0.7377
Epoch [1/20], Step [605/625], Loss: 1.3147
Epoch [1/20], Step [606/625], Loss: 0.5572
Epoch [1/20], Step [607/625], Loss: 0.5763
Epoch [1/20

Epoch [2/20], Step [155/625], Loss: 0.6032
Epoch [2/20], Step [156/625], Loss: 1.3222
Epoch [2/20], Step [157/625], Loss: 0.4323
Epoch [2/20], Step [158/625], Loss: 0.4549
Epoch [2/20], Step [159/625], Loss: 1.0824
Epoch [2/20], Step [160/625], Loss: 0.6240
Epoch [2/20], Step [161/625], Loss: 1.2647
Epoch [2/20], Step [162/625], Loss: 0.5368
Epoch [2/20], Step [163/625], Loss: 0.3780
Epoch [2/20], Step [164/625], Loss: 1.0156
Epoch [2/20], Step [165/625], Loss: 0.8079
Epoch [2/20], Step [166/625], Loss: 0.8396
Epoch [2/20], Step [167/625], Loss: 0.5434
Epoch [2/20], Step [168/625], Loss: 0.4982
Epoch [2/20], Step [169/625], Loss: 0.5506
Epoch [2/20], Step [170/625], Loss: 0.6182
Epoch [2/20], Step [171/625], Loss: 0.3985
Epoch [2/20], Step [172/625], Loss: 0.3790
Epoch [2/20], Step [173/625], Loss: 1.2498
Epoch [2/20], Step [174/625], Loss: 0.7017
Epoch [2/20], Step [175/625], Loss: 1.1903
Epoch [2/20], Step [176/625], Loss: 0.4795
Epoch [2/20], Step [177/625], Loss: 0.4676
Epoch [2/20

...

Epoch [19/20], Step [481/625], Loss: 0.3531
Epoch [19/20], Step [482/625], Loss: 0.1847
Epoch [19/20], Step [483/625], Loss: 0.4202
Epoch [19/20], Step [484/625], Loss: 0.0974
Epoch [19/20], Step [485/625], Loss: 0.2776
Epoch [19/20], Step [486/625], Loss: 0.0603
Epoch [19/20], Step [487/625], Loss: 0.1617
Epoch [19/20], Step [488/625], Loss: 0.1818
Epoch [19/20], Step [489/625], Loss: 0.0894
Epoch [19/20], Step [490/625], Loss: 0.2546
Epoch [19/20], Step [491/625], Loss: 0.2036
Epoch [19/20], Step [492/625], Loss: 0.1299
Epoch [19/20], Step [493/625], Loss: 0.2576
Epoch [19/20], Step [494/625], Loss: 0.1565
Epoch [19/20], Step [495/625], Loss: 0.1484
Epoch [19/20], Step [496/625], Loss: 0.0752
Epoch [19/20], Step [497/625], Loss: 0.1312
Epoch [19/20], Step [498/625], Loss: 0.3008
Epoch [19/20], Step [499/625], Loss: 0.0985
Epoch [19/20], Step [500/625], Loss: 0.1054
Epoch [19/20], Step [501/625], Loss: 0.1882
Epoch [19/20], Step [502/625], Loss: 0.2073
Epoch [19/20], Step [503/625], L

Epoch [20/20], Step [46/625], Loss: 0.0744
Epoch [20/20], Step [47/625], Loss: 0.0892
Epoch [20/20], Step [48/625], Loss: 0.1260
Epoch [20/20], Step [49/625], Loss: 0.1834
Epoch [20/20], Step [50/625], Loss: 0.1467
Epoch [20/20], Step [51/625], Loss: 0.1559
Epoch [20/20], Step [52/625], Loss: 0.2334
Epoch [20/20], Step [53/625], Loss: 0.1131
Epoch [20/20], Step [54/625], Loss: 0.1174
Epoch [20/20], Step [55/625], Loss: 0.1851
Epoch [20/20], Step [56/625], Loss: 0.2210
Epoch [20/20], Step [57/625], Loss: 0.3506
Epoch [20/20], Step [58/625], Loss: 0.1132
Epoch [20/20], Step [59/625], Loss: 0.1172
Epoch [20/20], Step [60/625], Loss: 0.0795
Epoch [20/20], Step [61/625], Loss: 0.1091
Epoch [20/20], Step [62/625], Loss: 0.1203
Epoch [20/20], Step [63/625], Loss: 0.1312
Epoch [20/20], Step [64/625], Loss: 0.1882
Epoch [20/20], Step [65/625], Loss: 0.0600
Epoch [20/20], Step [66/625], Loss: 0.3718
Epoch [20/20], Step [67/625], Loss: 0.2168
Epoch [20/20], Step [68/625], Loss: 0.1375
Epoch [20/2

Epoch [20/20], Step [236/625], Loss: 0.4086
Epoch [20/20], Step [237/625], Loss: 0.1471
Epoch [20/20], Step [238/625], Loss: 0.1876
Epoch [20/20], Step [239/625], Loss: 0.0975
Epoch [20/20], Step [240/625], Loss: 0.1586
Epoch [20/20], Step [241/625], Loss: 0.1596
Epoch [20/20], Step [242/625], Loss: 0.0708
Epoch [20/20], Step [243/625], Loss: 0.1228
Epoch [20/20], Step [244/625], Loss: 0.1380
Epoch [20/20], Step [245/625], Loss: 0.1821
Epoch [20/20], Step [246/625], Loss: 0.0954
Epoch [20/20], Step [247/625], Loss: 0.0723
Epoch [20/20], Step [248/625], Loss: 0.1415
Epoch [20/20], Step [249/625], Loss: 0.1121
Epoch [20/20], Step [250/625], Loss: 0.2259
Epoch [20/20], Step [251/625], Loss: 0.1112
Epoch [20/20], Step [252/625], Loss: 0.1695
Epoch [20/20], Step [253/625], Loss: 0.1550
Epoch [20/20], Step [254/625], Loss: 0.1169
Epoch [20/20], Step [255/625], Loss: 0.1971
Epoch [20/20], Step [256/625], Loss: 0.1034
Epoch [20/20], Step [257/625], Loss: 0.1073
Epoch [20/20], Step [258/625], L

Epoch [20/20], Step [426/625], Loss: 0.2626
Epoch [20/20], Step [427/625], Loss: 0.2087
Epoch [20/20], Step [428/625], Loss: 0.1865
Epoch [20/20], Step [429/625], Loss: 0.0728
Epoch [20/20], Step [430/625], Loss: 0.1858
Epoch [20/20], Step [431/625], Loss: 0.2032
Epoch [20/20], Step [432/625], Loss: 0.1528
Epoch [20/20], Step [433/625], Loss: 0.1149
Epoch [20/20], Step [434/625], Loss: 0.1360
Epoch [20/20], Step [435/625], Loss: 0.1956
Epoch [20/20], Step [436/625], Loss: 0.1928
Epoch [20/20], Step [437/625], Loss: 0.3033
Epoch [20/20], Step [438/625], Loss: 0.2744
Epoch [20/20], Step [439/625], Loss: 0.0861
Epoch [20/20], Step [440/625], Loss: 0.2010
Epoch [20/20], Step [441/625], Loss: 0.0904
Epoch [20/20], Step [442/625], Loss: 0.3211
Epoch [20/20], Step [443/625], Loss: 0.1788
Epoch [20/20], Step [444/625], Loss: 0.2186
Epoch [20/20], Step [445/625], Loss: 0.1369
Epoch [20/20], Step [446/625], Loss: 0.3981
Epoch [20/20], Step [447/625], Loss: 0.3218
Epoch [20/20], Step [448/625], L

Epoch [20/20], Step [616/625], Loss: 0.1115
Epoch [20/20], Step [617/625], Loss: 0.3401
Epoch [20/20], Step [618/625], Loss: 0.1229
Epoch [20/20], Step [619/625], Loss: 0.4818
Epoch [20/20], Step [620/625], Loss: 0.1848
Epoch [20/20], Step [621/625], Loss: 0.1199
Epoch [20/20], Step [622/625], Loss: 0.1238
Epoch [20/20], Step [623/625], Loss: 0.1325
Epoch [20/20], Step [624/625], Loss: 0.1804
Epoch [20/20], Step [625/625], Loss: 0.1100


In [39]:
best_epoch

8

In [40]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [41]:
G,P = total_labels, total_preds

In [42]:
rmse(G,P)

0.6378327844386453

In [43]:
mse(G,P)

0.40683066090475534

In [44]:
pearson(G,P)

0.5056825625107811

In [45]:
ci(G,P)

0.7785564118503862

In [46]:
model = ConvNet().to(device)

In [48]:
model.load_state_dict(torch.load('./best_sim-CNN-DTA_davis_fold0x.model'))

<All keys matched successfully>

In [49]:
model.eval()


ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 18, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=15390, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [50]:
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [51]:
G,P = total_labels, total_preds

In [52]:
rmse(G,P)

0.6140930322271506

In [53]:
print("MSE = ",mse(G,P))
print("R = ",pearson(G,P))
print("CI = ",ci(G,P))
print("RMSE = ",rmse(G,P))


MSE =  0.3771102522299362
R =  0.5336856702146883
CI =  0.78030908638589
RMSE =  0.6140930322271506
